# library loading

In [1]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time
import re # 정규표현식
import pandas as pd #csv저장

# information
- 상품명(pname), 총별점(tstar), 닉네임(name), 별점(star), 작성일(pdate), 기타리뷰(rv)
- pname_l , ..., non_l(리뷰없음), fail_l(경로가 다른지 안가져와진 애들)

In [2]:
# 화면 주소 정의
## 스킨케어
a = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010001'
## 마스크팩
b = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010009&gateCd=Drawer'
## 클렌징
c = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010010&gateCd=Drawer'
## 선케어
d = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010011&gateCd=Drawer'

# self function

In [3]:
def rv_num2():
    # 리뷰 수 //*[@id="gdasContentsArea"]/div/div[1]/div/div[2]/p[1]/em
    global q
    global pn
    global pc
    q = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text[1:-1]
    q = int(re.sub(r"[^0-9]", "", q))
    if q == 0:
        
        pn = 'zero'
        pc = 'zero'
    elif 0 < q <= 10:
        pn = 'zero'
        pc = q
    else:   
    # page_num(리뷰 페이지 수), product_count(마지막페이지의 리뷰수)
        pn = (q // 10) + 1 
        pc = q % 10

In [4]:
def rv_info2():
    try :
        global pname
        # //*[@id="Contents"]/div[2]/div[2]/div/p[2]
        # //*[@id="Contents"]/div[2]/div[2]/div/p[2]
        pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
        global tstar
        # #gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong
        tstar = driver.find_element(By.CSS_SELECTOR,'#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong').text
    except:
        print('별점은없어')
        pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text

In [5]:
def list_up(n=10):  # 파라미터는 한 페이지 안에 있는 리뷰수
    for i in range(1, n+1):
        pname_l.append(pname)
        tstar_l.append(tstar)
        # 닉네임
        # //*[@id="gdasList"]/li[1]/div[1]
        # //*[@id="gdasList"]/li[2]/div[1]/div
        # //*[@id="gdasList"]/li[2]/div[1]/div/p/a
        y = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]').text
        name = y.split(sep='\n')[0]
        name_l.append(name)
        time.sleep(0.5) # x에서 계속 멈춰서 1일떈 끝까지 감
        # 리뷰 블럭 //*[@id="gdasList"]/li[4]/div[2]
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]').text
        rv = x.split(sep='\n')
        ## 별점
        star = rv[0]
        star_l.append(star)
        ## 작성일
        pdate = rv[1]
        pdate_l.append(pdate)
        ##기타
        del rv[0:2] 
        del rv[-2:]
        rv_l.append(rv)
        print(f'_{i}') 

In [6]:
def one_product2(pn, pc):
    list_up()
    for l in range(2, pn+1): 
        if str(l)[-1] == '0':    
            try: # 10페이지가 마지막인 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click()
                driver.implicitly_wait(3)
                # 10개의 정보 가져오는 함수 기본함수정의해서파라미터로 숫자넣기(10) 
                print(l)
                list_up()
                driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
                driver.implicitly_wait(3)
                print(l)
                list_up()
            except:
                driver.find_element(By.LINK_TEXT, f'{l}').click()
                driver.implicitly_wait(3)
                # 10개의 정보 가져오는 함수 기본함수정의해서파라미터로 숫자넣기(10) 
                list_up(pc)
                print(l, 'complete')
                
            
            # 함수가져오기 list_up()
        elif str(l)[-1] == '0' and l == pn: # 다음페이지가 마지막인 경우
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            list_up()
            driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
            driver.implicitly_wait(3)
            print(l)
            if pc != 0: # pc가 0이면 for문 안돌아가서 
                list_up(pc)
                print('수집완료^^')
            else:
                list_up()
                print('수집완료^')
        elif str(l)[-1] == '1':
            print(l)
        elif l == (pn): # 마지막 페이지
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            if pc != 0:
                list_up(pc)
                print('수집완료^^')
            else:
                list_up(pc)
                print('수집완??')
            # 끝자리수만큼 포문 pc list_up(pc)
            print(l)
        else:
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            list_up()
            print(l)

In [7]:
def try_pro():
    try:
        rv_num2()
        rv_info2()
        if pn == 'zero' and pc == 'zero':
            # 상품리뷰 없음. 
            non_l.append(pname) # 상품명 리스트에는 안올라감
        elif pn == 'zero':
            list_up(pc)
            print('출력완료^^')
        else:
            one_product2(pn, pc)
    except:
        pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
        fail_l.append(pname)
        print("==== 문제 발생 ====", pname)

## do it

In [8]:
#  driver setting
URL = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0' # 첫화면
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])

# chromedriver path
sv = Service(executable_path= 'C:\\Users\\admin\\Downloads\\chroemdriver')
driver = webdriver.Chrome(service=sv, options=ops)

In [9]:
# page moving
driver.get(url=a)
driver.implicitly_wait(3)
# 기본페이지 이동
# 스킨케어_토너 페이지로 이동
driver.find_element(By.XPATH, f'//*[@id="Contents"]/div/div[1]/ul/li[1]/a').click()
driver.implicitly_wait(3)
# 리뷰 48개로 선택
driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click()
driver.implicitly_wait(3)

In [10]:
# 리스트 정의
pname_l = []
tstar_l = []
name_l = []
star_l = []
pdate_l = []
rv_l = []

non_l = []
fail_l = []

In [11]:
# crawler 
# for문 가능?
for x in range(2,7):
    try:
        for i in range(1, 13):
            for j in range(1,5):
                # 상품 선택
                driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                time.sleep(3)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                time.sleep(3)
                try_pro()
                driver.back()
    except:
        print('어디서 오류가 났으려나? 일단 pass^^')
    try:
        driver.find_element(By.LINK_TEXT, f'{10+x}').click()
        driver.implicitly_wait(3)
    except:
        continue

_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
2
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
3
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
4
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
5
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
6
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
7
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
8
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
9
10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
11
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
12
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
13
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
14
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
15
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
16
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
17
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
18
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
19
20
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
20
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
21
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
22
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
23
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
24
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
25
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
26
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
27
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
28
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
29
30
_1
_2
_3
_4
_5
_

In [12]:
# restore data
try:   
    rv_rv = pd.DataFrame({'pname':pname_l, 'tstar':tstar_l, 'name':name_l,\
                          'star':star_l,'pdate':pdate_l, 'rv':rv_l})
    rv_rv.to_csv('./newtry_rvVer2.csv', sep = ',', encoding = 'utf-8-sig')
except:
    rv_rv = pd.DataFrame({'star':star_l,'pdate':pdate_l, 'rv':rv_l})
    rv_rv.to_csv('./newtry_rvVer2.csv', sep = ',', encoding = 'utf-8-sig')
    rv_name = pd.DataFrame({'pname':pname_l, 'tstar':tstar_l})
    rv_name.to_csv('./newtry_nameVer2.csv', sep = ',', encoding = 'utf-8-sig')
    print('this')

this


In [13]:
rv_zero = pd.DataFrame({'pname':non_l})
rv_zero.to_csv('./newtry_zeroVer2.csv', sep = ',', encoding = 'utf-8-sig')
rv_fail = pd.DataFrame({'pname':fail_l})
rv_fail.to_csv('./newtry_failVer2.csv', sep = ',', encoding = 'utf-8-sig')

In [14]:
# exit
driver.close()

KeyboardInterrupt: 